In [15]:
import os

# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_CNRM-ESM2-1_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\WRF_CESM2_ssp370_BurnRasters\late
Successfully changed directory to: I:\loca2\WRF_CNRM-ESM2-1_ssp370_BurnRasters


In [11]:
import rasterio
import numpy as np
import os

def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_CESM2_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "averaged_geotiff.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 30 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[  0.         0.        23.724884  25.5       60.409573]
 [  0.        23.724884  25.5       60.409573  94.83156 ]
 [  0.        23.724884  38.980446  72.42913   94.83156 ]
 [  0.        37.20533   72.42913   76.5      119.83187 ]
 [ 13.480444  49.224888  72.42913  101.50031  131.60663 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [17]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/CNRM-ESM2-1_ssp370_r1i1p1f2/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/CNRM-ESM2-1_ssp370_r1i1p1f2/WesterlingFireModel/BurnRasters/
Found 118 TIFF files to download.
Successfully downloaded burned_3km_TA_1982.tif
Successfully downloaded burned_3km_TA_1983.tif
Successfully downloaded burned_3km_TA_1984.tif
Successfully downloaded burned_3km_TA_1985.tif
Successfully downloaded burned_3km_TA_1986.tif
Successfully downloaded burned_3km_TA_1987.tif
Successfully downloaded burned_3km_TA_1988.tif
Successfully downloaded burned_3km_TA_1989.tif
Successfully downloaded burned_3km_TA_1990.tif
Successfully downloaded burned_3km_TA_1991.tif
Successfully downloaded burned_3km_TA_1992.tif
Successfully downloaded burned_3km_TA_1993.tif
Successfully downloaded burned_3km_TA_1994.tif
Successfully downloaded burned_3km_TA_1995.tif
Successfully downloaded burned_3km_TA_1996.tif
Successfully downloaded burned_3km_TA_1997.tif
Successfully downloa

In [19]:
import os

# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_EC-Earth3-Veg_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\WRF_CNRM-ESM2-1_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_EC-Earth3-Veg_ssp370_BurnRasters


In [21]:

import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/EC-Earth3-Veg_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/EC-Earth3-Veg_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/
Found 118 TIFF files to download.
Successfully downloaded burned_3km_TA_1982.tif
Successfully downloaded burned_3km_TA_1983.tif
Successfully downloaded burned_3km_TA_1984.tif
Successfully downloaded burned_3km_TA_1985.tif
Successfully downloaded burned_3km_TA_1986.tif
Successfully downloaded burned_3km_TA_1987.tif
Successfully downloaded burned_3km_TA_1988.tif
Successfully downloaded burned_3km_TA_1989.tif
Successfully downloaded burned_3km_TA_1990.tif
Successfully downloaded burned_3km_TA_1991.tif
Successfully downloaded burned_3km_TA_1992.tif
Successfully downloaded burned_3km_TA_1993.tif
Successfully downloaded burned_3km_TA_1994.tif
Successfully downloaded burned_3km_TA_1995.tif
Successfully downloaded burned_3km_TA_1996.tif
Successfully downloaded burned_3km_TA_1997.tif
Successfully downl

In [22]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_FGOALS-g3_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\WRF_EC-Earth3-Veg_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_FGOALS-g3_ssp370_BurnRasters


In [23]:

import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/FGOALS-g3_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/FGOALS-g3_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/
Found 118 TIFF files to download.
Successfully downloaded burned_3km_TA_1982.tif
Successfully downloaded burned_3km_TA_1983.tif
Successfully downloaded burned_3km_TA_1984.tif
Successfully downloaded burned_3km_TA_1985.tif
Successfully downloaded burned_3km_TA_1986.tif
Successfully downloaded burned_3km_TA_1987.tif
Successfully downloaded burned_3km_TA_1988.tif
Successfully downloaded burned_3km_TA_1989.tif
Successfully downloaded burned_3km_TA_1990.tif
Successfully downloaded burned_3km_TA_1991.tif
Successfully downloaded burned_3km_TA_1992.tif
Successfully downloaded burned_3km_TA_1993.tif
Successfully downloaded burned_3km_TA_1994.tif
Successfully downloaded burned_3km_TA_1995.tif
Successfully downloaded burned_3km_TA_1996.tif
Successfully downloaded burned_3km_TA_1997.tif
Successfully downloade

In [27]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_CESM2_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\WRF_FGOALS-g3_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_CESM2_ssp370_BurnRasters


In [29]:

import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/CESM2_ssp370_r11i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/wrf_bc/CESM2_ssp370_r11i1p1f1/WesterlingFireModel/BurnRasters/
Found 118 TIFF files to download.
Successfully downloaded burned_3km_TA_1982.tif
Successfully downloaded burned_3km_TA_1983.tif
Successfully downloaded burned_3km_TA_1984.tif
Successfully downloaded burned_3km_TA_1985.tif
Successfully downloaded burned_3km_TA_1986.tif
Successfully downloaded burned_3km_TA_1987.tif
Successfully downloaded burned_3km_TA_1988.tif
Successfully downloaded burned_3km_TA_1989.tif
Successfully downloaded burned_3km_TA_1990.tif
Successfully downloaded burned_3km_TA_1991.tif
Successfully downloaded burned_3km_TA_1992.tif
Successfully downloaded burned_3km_TA_1993.tif
Successfully downloaded burned_3km_TA_1994.tif
Successfully downloaded burned_3km_TA_1995.tif
Successfully downloaded burned_3km_TA_1996.tif
Successfully downloaded burned_3km_TA_1997.tif
Successfully downloaded b

In [30]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_ACCESS-CM2_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\WRF_CESM2_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_ACCESS-CM2_ssp370_BurnRasters


In [33]:

import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/ACCESS-CM2_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/ACCESS-CM2_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/
Found 86 TIFF files to download.
Successfully downloaded burned_3km_TA_2015.tif
Successfully downloaded burned_3km_TA_2016.tif
Successfully downloaded burned_3km_TA_2017.tif
Successfully downloaded burned_3km_TA_2018.tif
Successfully downloaded burned_3km_TA_2019.tif
Successfully downloaded burned_3km_TA_2020.tif
Successfully downloaded burned_3km_TA_2021.tif
Successfully downloaded burned_3km_TA_2022.tif
Successfully downloaded burned_3km_TA_2023.tif
Successfully downloaded burned_3km_TA_2024.tif
Successfully downloaded burned_3km_TA_2025.tif
Successfully downloaded burned_3km_TA_2026.tif
Successfully downloaded burned_3km_TA_2027.tif
Successfully downloaded burned_3km_TA_2028.tif
Successfully downloaded burned_3km_TA_2029.tif
Successfully downloaded burned_3km_TA_2030.tif
Successfully downloaded

In [34]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_EC-Earth3-Veg_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\LOCA2_ACCESS-CM2_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_EC-Earth3-Veg_ssp370_BurnRasters


In [35]:

import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin, urlparse

def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/EC-Earth3-Veg_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/EC-Earth3-Veg_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/
Found 100 TIFF files to download.
Successfully downloaded burned_3km_TA_2000.tif
Successfully downloaded burned_3km_TA_2001.tif
Successfully downloaded burned_3km_TA_2002.tif
Successfully downloaded burned_3km_TA_2003.tif
Successfully downloaded burned_3km_TA_2004.tif
Successfully downloaded burned_3km_TA_2005.tif
Successfully downloaded burned_3km_TA_2006.tif
Successfully downloaded burned_3km_TA_2007.tif
Successfully downloaded burned_3km_TA_2008.tif
Successfully downloaded burned_3km_TA_2009.tif
Successfully downloaded burned_3km_TA_2010.tif
Successfully downloaded burned_3km_TA_2011.tif
Successfully downloaded burned_3km_TA_2012.tif
Successfully downloaded burned_3km_TA_2013.tif
Successfully downloaded burned_3km_TA_2014.tif
Successfully downloaded burned_3km_TA_2016.tif
Successfully downlo

In [36]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_MIROC6_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\LOCA2_EC-Earth3-Veg_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_MIROC6_ssp370_BurnRasters


In [37]:
def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/MIROC6_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/MIROC6_ssp370_r1i1p1f1/WesterlingFireModel/BurnRasters/
Found 86 TIFF files to download.
Successfully downloaded burned_3km_TA_2015.tif
Successfully downloaded burned_3km_TA_2016.tif
Successfully downloaded burned_3km_TA_2017.tif
Successfully downloaded burned_3km_TA_2018.tif
Successfully downloaded burned_3km_TA_2019.tif
Successfully downloaded burned_3km_TA_2020.tif
Successfully downloaded burned_3km_TA_2021.tif
Successfully downloaded burned_3km_TA_2022.tif
Successfully downloaded burned_3km_TA_2023.tif
Successfully downloaded burned_3km_TA_2024.tif
Successfully downloaded burned_3km_TA_2025.tif
Successfully downloaded burned_3km_TA_2026.tif
Successfully downloaded burned_3km_TA_2027.tif
Successfully downloaded burned_3km_TA_2028.tif
Successfully downloaded burned_3km_TA_2029.tif
Successfully downloaded burned_3km_TA_2030.tif
Successfully downloaded bur

In [40]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")


Current directory: I:\loca2\LOCA2_MIROC6_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters


In [41]:
def download_tif_files_from_web_directory(url, download_dir="downloaded_tiffs"):
    """
    Downloads all .tif and .tiff files from a given web directory URL.

    Args:
        url (str): The URL of the web directory.
        download_dir (str): The local directory where files will be saved.
                            Defaults to "downloaded_tiffs".
    """
    print(f"Attempting to download TIFF files from: {url}")

    # Create the download directory if it doesn't exist
    os.makedirs(download_dir, exist_ok=True)

    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
    except requests.exceptions.RequestException as e:
        print(f"Error accessing the URL {url}: {e}")
        return

    soup = BeautifulSoup(response.text, 'html.parser')
    
    tif_links = []
    # Find all <a> tags (links)
    for link in soup.find_all('a'):
        href = link.get('href')
        if href:
            # Check if the link ends with .tif or .tiff (case-insensitive)
            if href.lower().endswith(('.tif', '.tiff')):
                full_url = urljoin(url, href) # Construct full URL
                tif_links.append(full_url)
    
    if not tif_links:
        print(f"No .tif or .tiff files found at {url}. This might be because:")
        print("- The directory does not contain .tif files.")
        print("- The directory listing format is not standard HTML (e.g., JSON API, custom server configuration).")
        print("- There are no visible links to .tif files in the HTML.")
        return

    print(f"Found {len(tif_links)} TIFF files to download.")
    for tif_url in tif_links:
        file_name = os.path.basename(urlparse(tif_url).path)
        local_filepath = os.path.join(download_dir, file_name)

        print(f"Downloading {file_name} to {local_filepath}...")
        try:
            with requests.get(tif_url, stream=True) as r:
                r.raise_for_status()
                with open(local_filepath, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192):
                        f.write(chunk)
            print(f"Successfully downloaded {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {file_name} from {tif_url}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred while saving {file_name}: {e}")

# --- Example Usage ---
if __name__ == "__main__":
    # Replace with the actual URL of the web directory containing .tif files
    # This example URL is hypothetical. You would need to find a real one.
    # A common scenario is public data repositories or FTP/HTTP servers.

    # Example of a public data repository with .tif files (replace with a real one)
    # Be mindful of the terms of service and download policies of the site you are accessing.
    # For demonstration, I'll use a placeholder URL.
    # You'll need to find a URL that actually lists TIFF files directly in its HTML.
    
    # A common format for directory listings is Apache's default directory index,
    # which lists files as hyperlinks.
    
    # Example (hypothetical, as I cannot browse the web directly):
    # Consider a URL like "http://example.com/data/images/"
    # where the HTML content directly contains links such as:
    # <a href="image1.tif">image1.tif</a>
    # <a href="image2.tiff">image2.tiff</a>
    
    # For a realistic test, you might need to find an open directory listing.
    # Some public data servers (e.g., NOAA, NASA, some university servers) might have them.
    # Always check the robots.txt and terms of use for any site you scrape.

    # example_url = "http://www.unidata.ucar.edu/software/netcdf/examples/files/" # This URL might not contain .tif, used for structure
    # A more likely scenario for a .tif directory might be a specific dataset folder, e.g.:
    # example_url = "https://www.ngdc.noaa.gov/dem/square/501300_500100_GEOTIFF/" 
    # (Note: The NOAA URL above has a directory listing but often contains other files too,
    # and direct links might require specific parsing if not straightforward <a> tags.)

    # For a simple local test if you have a web server set up
    # or just want to see the code logic with a mock scenario:
    # 1. Create a simple HTML file (e.g., `test_dir.html`) with links to dummy .tif files:
    #    <html><body><a href="dummy1.tif">dummy1.tif</a><a href="dummy2.tiff">dummy2.tiff</a></body></html>
    # 2. Put some dummy .tif files in the same directory.
    # 3. Serve that directory locally with Python's http.server: `python -m http.server 8000`
    # 4. Then `example_url = "http://localhost:8000/test_dir.html"`
    
    print("\n--- Starting Download Process ---")
    # Please replace this with an actual URL that serves a directory listing with .tif files
    # For the purpose of providing runnable code, I'll use a URL that's known to exist
    # but likely won't have .tif files directly in its listing, so the output will reflect that.
    # You will need to find a suitable URL for your specific use case.
    
    # A *more robust* approach for complex web directories might involve deeper inspection
    # of link patterns or API calls if the directory isn't a simple HTML listing.
    
    download_tif_files_from_web_directory("http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/MPI-ESM1-2-HR_ssp370_r3i1p1f1/WesterlingFireModel/BurnRasters/") 
    # This NOAA URL provides a good example of an actual web directory
    # containing .tif files. Note: This specific path might change or require
    # further navigation depending on NOAA's site structure.
    
    print("\n--- Download Process Completed ---")


--- Starting Download Process ---
Attempting to download TIFF files from: http://ungoliant.ucmerced.edu/data/CEC-Preliminary/loca2/MPI-ESM1-2-HR_ssp370_r3i1p1f1/WesterlingFireModel/BurnRasters/
Found 86 TIFF files to download.
Successfully downloaded burned_3km_TA_2015.tif
Successfully downloaded burned_3km_TA_2016.tif
Successfully downloaded burned_3km_TA_2017.tif
Successfully downloaded burned_3km_TA_2018.tif
Successfully downloaded burned_3km_TA_2019.tif
Successfully downloaded burned_3km_TA_2020.tif
Successfully downloaded burned_3km_TA_2021.tif
Successfully downloaded burned_3km_TA_2022.tif
Successfully downloaded burned_3km_TA_2023.tif
Successfully downloaded burned_3km_TA_2024.tif
Successfully downloaded burned_3km_TA_2025.tif
Successfully downloaded burned_3km_TA_2026.tif
Successfully downloaded burned_3km_TA_2027.tif
Successfully downloaded burned_3km_TA_2028.tif
Successfully downloaded burned_3km_TA_2029.tif
Successfully downloaded burned_3km_TA_2030.tif
Successfully downloa

In [78]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_ACCESS-CM2_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_ACCESS-CM2_ssp370_BurnRasters


In [80]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_ACCESS-CM2_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_ACCESS-CM2_ssp370_BurnRasters/LOCA2_ACCESS-CM2_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 51.        73.868546  76.5       89.61656   89.61656 ]
 [ 73.868546  76.5       76.5       89.61656  102.      ]
 [ 73.868546  76.5       89.61656   89.61656  102.      ]
 [ 76.5       76.5       89.61656  102.       102.      ]
 [ 76.5       76.5       89.61656  102.       102.      ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [82]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_ACCESS-CM2_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_ACCESS-CM2_ssp370_BurnRasters/LOCA2_ACCESS-CM2_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 51.        56.006508  84.51991  121.37629  178.5     ]
 [ 51.        64.02641  102.       121.37629  198.45525 ]
 [ 51.        64.02641  103.10309  145.77318  198.45525 ]
 [ 59.019905  81.506516 103.10309  165.72845  221.41252 ]
 [ 59.019905  81.506516 103.10309  165.72845  233.37047 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [86]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_EC-Earth3-Veg_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\LOCA2_EC-Earth3-Veg_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_EC-Earth3-Veg_ssp370_BurnRasters


In [88]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_EC-Earth3-Veg_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_EC-Earth3-Veg_ssp370_BurnRasters/LOCA2_Earth3-Veg_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 76.5      88.89495 102.      102.      116.61429]
 [ 76.5      88.89495 102.      116.61429 127.5    ]
 [ 76.5      88.89495 102.      116.61429 127.5    ]
 [ 76.5      88.89495 102.      116.61429 127.5    ]
 [ 76.5      88.89495 102.      116.61429 127.5    ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [90]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_EC-Earth3-Veg_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_EC-Earth3-Veg_ssp370_BurnRasters/LOCA2_Earth3-Veg_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[163.26773 209.68535 231.44466 270.29596 319.84482]
 [176.97119 211.62997 231.44466 294.98996 366.73132]
 [184.46368 211.62997 270.29596 305.19403 371.4013 ]
 [184.46368 211.62997 270.29596 305.19403 371.4013 ]
 [186.4083  235.18535 270.29596 309.86395 395.05768]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [92]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_MIROC6_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\LOCA2_EC-Earth3-Veg_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_MIROC6_ssp370_BurnRasters


In [94]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_MIROC6_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_MIROC6_ssp370_BurnRasters/LOCA2_MIROC6_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[0.        0.        0.        0.        0.       ]
 [0.        0.        0.        0.        0.       ]
 [0.        0.        0.        0.        5.7259517]
 [0.        0.        0.        0.        5.7259517]
 [0.        0.        0.        0.        5.7259517]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [96]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_MIROC6_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_MIROC6_ssp370_BurnRasters/LOCA2_MIROC6_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[25.5      25.5      25.5      25.5      25.5     ]
 [25.5      25.5      25.5      25.5      25.525888]
 [25.5      25.5      25.5      25.5      25.525888]
 [25.5      25.5      25.5      25.525888 51.      ]
 [25.5      25.5      25.5      25.525888 71.68896 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [98]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\LOCA2_MIROC6_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters


In [100]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters/LOCA2_MPI-ESM1-2-HR_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 32.573803  51.        69.73076  111.35825  135.81099 ]
 [ 32.573803  69.73076   69.73076  111.35825  138.8366  ]
 [ 51.        69.73076   97.85453  115.50372  138.8366  ]
 [ 51.        69.73076   97.85453  118.529335 161.31099 ]
 [ 69.73076   97.85453  102.       118.529335 161.31099 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [102]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters/LOCA2_MPI-ESM1-2-HR_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[25.5     25.5     25.5     25.5     46.90264]
 [25.5     25.5     25.5     25.5     46.90264]
 [25.5     25.5     25.5     46.90264 68.96124]
 [25.5     25.5     25.5     46.90264 68.96124]
 [25.5     25.5     25.5     46.90264 68.96124]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_paralle

In [106]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_CESM2_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\LOCA2_MPI-ESM1-2-HR_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_CESM2_ssp370_BurnRasters


In [108]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_CESM2_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_CESM2_ssp370_BurnRasters/WRF_CESM2_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 31 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[25.5 25.5 25.5 25.5 25.5]
 [25.5 25.5 25.5 25.5 25.5]
 [25.5 25.5 25.5 25.5 25.5]
 [25.5 25.5 25.5 25.5 25.5]
 [25.5 25.5 25.5 25.5 25.5]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PARAMETER["standard_parallel_2",40.5],PARAMETER["false_easting",0],PARAMETER["false_northing",-4000000],UNIT["metre",1,AUTHORIT

In [110]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_CESM2_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_CESM2_ssp370_BurnRasters/WRF_CESM2_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 30 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[  0.         0.        24.515715  26.35      62.423225]
 [  0.        24.515715  26.35      62.423225  97.99261 ]
 [  0.        24.515715  40.279793  74.84344   97.99261 ]
 [  0.        38.445507  74.84344   79.05     123.82626 ]
 [ 13.929792  50.865715  74.84344  104.88365  135.99352 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [112]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_CNRM-ESM2-1_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\WRF_CESM2_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_CNRM-ESM2-1_ssp370_BurnRasters


In [114]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_CNRM-ESM2-1_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_CNRM-ESM2-1_ssp370_BurnRasters/WRF_CNRM-ESM2-1_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 30 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 76.5      76.5      76.5      92.15157 102.     ]
 [ 76.5      76.5      92.15157 102.      102.     ]
 [ 76.5      76.5      92.15157 102.      102.     ]
 [ 76.5      92.15157 102.      102.      102.     ]
 [ 76.5      92.15157 102.      102.      102.     ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [116]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_CNRM-ESM2-1_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_CNRM-ESM2-1_ssp370_BurnRasters/WRF_CNRM-ESM2-1_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 29 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[100.92016 118.84976 158.27586 164.14507 184.65517]
 [105.51724 118.84976 164.14507 184.65517 196.83253]
 [106.86249 143.88382 164.14507 184.65517 214.10417]
 [106.86249 143.88382 164.14507 184.65517 219.91422]
 [106.86249 143.88382 164.14507 184.65517 237.67776]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [118]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_EC-Earth3-Veg_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\WRF_CNRM-ESM2-1_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_EC-Earth3-Veg_ssp370_BurnRasters


In [120]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_EC-Earth3-Veg_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_EC-Earth3-Veg_ssp370_BurnRasters/WRF_EC-Earth3-Veg_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 30 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[102.588974 127.5      127.5      127.5      127.5     ]
 [102.588974 127.5      127.5      127.5      152.38509 ]
 [127.5      127.5      127.5      127.5      152.38509 ]
 [127.5      127.5      127.5      127.5      152.38509 ]
 [127.5      127.5      127.5      127.5      152.38509 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [122]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_EC-Earth3-Veg_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_EC-Earth3-Veg_ssp370_BurnRasters/WRF_EC-Earth3-Veg_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 29 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[131.89655 131.89655 156.76967 187.35313 231.81699]
 [131.89655 131.89655 173.4749  211.55374 289.8825 ]
 [131.89655 131.89655 179.73087 211.55374 289.8825 ]
 [131.89655 131.89655 179.73087 242.32787 331.99033]
 [131.89655 131.89655 179.73087 242.32787 331.99033]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [124]:
# Get the current working directory (optional)
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Define the new directory path
new_directory = "I:/loca2/WRF_FGOALS-g3_ssp370_BurnRasters"  # Replace with your desired path

# Change the directory
try:
    os.chdir(new_directory)
    print(f"Successfully changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print(f"Error: The directory '{new_directory}' was not found.")
except Exception as e:
    print(f"An error occurred: {e}")

Current directory: I:\loca2\WRF_EC-Earth3-Veg_ssp370_BurnRasters
Successfully changed directory to: I:\loca2\WRF_FGOALS-g3_ssp370_BurnRasters


In [126]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_FGOALS-g3_ssp370_BurnRasters/mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_FGOALS-g3_ssp370_BurnRasters/WRF_FGOALS-g3_ssp370_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 30 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[51.436634 76.5      76.5      76.5      76.5     ]
 [51.436634 76.5      76.5      76.5      76.5     ]
 [51.436634 76.5      76.5      76.5      76.5     ]
 [51.436634 76.5      76.5      76.5      76.5     ]
 [51.436634 76.5      76.5      76.5      76.5     ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PA

In [128]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/WRF_FGOALS-g3_ssp370_BurnRasters/late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/WRF_FGOALS-g3_ssp370_BurnRasters/WRF_FGOALS-g3_ssp370_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 29 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 20.40594   26.37931   61.416115 125.66324  132.18983 ]
 [ 20.40594   41.186752  99.164795 125.86574  158.27586 ]
 [ 20.40594   41.186752 119.51329  132.18983  176.7428  ]
 [ 20.40594   67.363556 119.7158   158.27586  176.7428  ]
 [ 35.21338   78.93543  119.7158   176.7428   249.40608 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["st

In [140]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/Pyregence/LOCA2_mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/Pyregence/Pyregence_LOCA2_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 5 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[40.01845  53.44087  62.05769  75.743706 85.51046 ]
 [45.735588 58.781425 62.05769  79.39728  92.08415 ]
 [50.342136 58.781425 72.367775 80.43364  93.51564 ]
 [51.       58.781425 72.367775 84.285904 99.13423 ]
 [55.68269  65.81237  73.404144 84.285904 99.13423 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PAR

In [142]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/Pyregence/LOCA2_late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/Pyregence/Pyregence_LOCA2_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 5 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 66.31693   79.172966  91.74114  110.66807  142.68686 ]
 [ 69.7428    81.66409   96.11116  116.84157  159.40378 ]
 [ 71.61592   81.66409  106.099754 130.84247  166.08592 ]
 [ 73.620895  86.03412  106.099754 135.83775  178.19377 ]
 [ 74.10705   91.922966 106.099754 137.00523  192.26959 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["sta

In [144]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/Pyregence/WRF_mid"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/Pyregence/Pyregence_WRF_2041-2070.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 5 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[64.0064   76.5      76.5      80.412895 82.875   ]
 [64.0064   76.5      80.412895 82.875    89.09627 ]
 [70.23416  76.5      80.412895 82.875    89.09627 ]
 [70.23416  80.412895 82.875    82.875    89.09627 ]
 [70.23416  80.412895 82.875    82.875    89.09627 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["standard_parallel_1",34],PAR

In [146]:
def average_geotiffs(directory_path):
    """
    Loads all GeoTIFF files from a directory, stacks them into an array,
    and calculates the average for each lat/lon location.

    Args:
        directory_path (str): The path to the directory containing GeoTIFF files.

    Returns:
        tuple: A tuple containing:
            - numpy.ndarray: The 2D array of averaged pixel values.
            - rasterio.transform.Affine: The affine transform for the averaged data.
            - dict: The metadata from the first GeoTIFF file (for projection, etc.).
            - list: A list of file paths that were processed.
    """
    geotiff_files = [os.path.join(directory_path, f) for f in os.listdir(directory_path) if f.lower().endswith(('.tif', '.tiff'))]

    if not geotiff_files:
        print(f"No GeoTIFF files found in the directory: {directory_path}")
        return None, None, None, []

    # Initialize an empty list to store data from each GeoTIFF
    data_list = []
    profile = None
    transform = None
    processed_files = []

    for i, filepath in enumerate(geotiff_files):
        try:
            with rasterio.open(filepath) as src:
                if i == 0:  # Get profile and transform from the first file
                    profile = src.profile
                    transform = src.transform
                    height = src.height
                    width = src.width
                
                # Ensure all rasters have the same dimensions
                if src.height != height or src.width != width:
                    print(f"Warning: Skipping {filepath} due to dimension mismatch. Expected ({height}, {width}), got ({src.height}, {src.width}).")
                    continue

                data_list.append(src.read(1))  # Read the first band
                processed_files.append(filepath)
        except rasterio.errors.RasterioIOError as e:
            print(f"Error reading {filepath}: {e}")
            continue

    if not data_list:
        print("No valid GeoTIFF files were processed.")
        return None, None, None, []

    # Stack all the 2D arrays into a 3D array
    stacked_data = np.stack(data_list, axis=0)

    # Calculate the average along the first axis (across all files)
    averaged_data = np.mean(stacked_data, axis=0)

    return averaged_data, transform, profile, processed_files

# --- Example Usage ---
if __name__ == "__main__":
    dummy_dir = "I:/loca2/Pyregence/WRF_late"
    averaged_array, avg_transform, avg_profile, processed_files_list = average_geotiffs(dummy_dir)

    if averaged_array is not None:
        print(f"\nSuccessfully processed {len(processed_files_list)} GeoTIFF files.")
        print("Shape of the averaged array:", averaged_array.shape)
        print("First 5x5 block of averaged data:\n", averaged_array[:5, :5])
        print("\nProfile of the averaged data (from first file):", avg_profile)
        print("Transform of the averaged data (from first file):", avg_transform)

        # Optionally, save the averaged GeoTIFF
        output_averaged_file = os.path.join(dummy_dir, "I:/loca2/Pyregence/Pyregence_WRF_2071-2099.tif")
        if avg_profile:
            avg_profile.update(dtype=averaged_array.dtype, count=1)
            with rasterio.open(output_averaged_file, 'w', **avg_profile) as dst:
                dst.write(averaged_array, 1)
            print(f"\nAveraged GeoTIFF saved to: {output_averaged_file}")

    # Clean up dummy files
    # import shutil
    # shutil.rmtree(dummy_dir)
    # print(f"\nCleaned up dummy directory: {dummy_dir}")


Successfully processed 5 GeoTIFF files.
Shape of the averaged array: (360, 310)
First 5x5 block of averaged data:
 [[ 63.305656  69.2814   100.24434  125.87787  152.7713  ]
 [ 64.45493   79.1122   115.78369  146.12448  185.74588 ]
 [ 64.791245  85.37071  125.91726  150.81055  194.68053 ]
 [ 64.791245  95.397354 134.6088   166.07722  213.11841 ]
 [ 71.975555 101.39538  134.6088   177.15237  238.76694 ]]

Profile of the averaged data (from first file): {'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.3999999521443642e+38, 'width': 310, 'height': 360, 'count': 1, 'crs': CRS.from_wkt('PROJCS["unknown",GEOGCS["unknown",DATUM["Unknown based on GRS 1980 ellipsoid using towgs84=0,0,0,0,0,0,0",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],TOWGS84[0,0,0,0,0,0,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",-120],PARAMETER["sta